In [1]:
from utils.dataloader import H5Dataset
from utils.EarlyStopping import EarlyStopping

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import classification_report
from tqdm import tqdm

from torchvision.models import MobileNet_V2_Weights

import numpy as np
import os

In [2]:
num_classes = 4
k_folds = 5
num_epochs = 10
batch_size = 32
patience = 2
random_state = 21
checkpoint_folder = "mobilenet"

# Caminho para os arquivos h5
h5_files = [
    "h5_files/GunshotForensic_feature.h5",
    "h5_files/HEAR_Train_feature.h5",
    "h5_files/MIVIA_test_feature.h5",
    "h5_files/MIVIA_train_feature.h5",
    "h5_files/SESA_feature.h5",
    "h5_files/vsd_clipped_feature.h5",
    "h5_files/Audioset_feature.h5",
]

# Carregando os dataloaders
dataset = H5Dataset(
    h5_files, exclude_classes=[4, 5, 6], n_splits=k_folds, random_state=random_state
)
dataloaders = dataset.get_k_fold_data_loaders(batch_size=batch_size, num_workers=4)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Treinamento realizado no dispositivo:", device)

criterion = nn.CrossEntropyLoss()

Treinamento realizado no dispositivo: cuda


In [4]:
# Loop de treinamento
for fold, (train_loader, val_loader) in enumerate(dataloaders):
    print(f"Starting fold {fold + 1}/{len(dataloaders)}")

    # Ajustando a MobileNetV2
    model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    model.features[0][0] = nn.Conv2d(
        1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
    )

    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True, fold=fold, checkpoint_folder=checkpoint_folder)

    for epoch in range(num_epochs):  # número de épocas
        print(f"Starting epoch {epoch + 1}/{num_epochs}")

        # Treinamento
        model.train()
        running_loss_train = 0.0
        running_corrects_train = 0
        for i, (inputs, labels) in tqdm(
            enumerate(train_loader), total=len(train_loader)
        ):
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects_train += torch.sum(preds == labels.data)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss_train += loss.item() * inputs.size(0)

        epoch_loss_train = running_loss_train / len(train_loader.dataset)
        epoch_acc_train = running_corrects_train.double() / len(train_loader.dataset)

        # Validação
        model.eval()
        running_loss_val = 0.0
        running_corrects_val = 0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs = inputs.unsqueeze(1).to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                running_corrects_val += torch.sum(preds == labels.data)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item() * inputs.size(0)

                all_labels.extend(labels.data.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        epoch_loss_val = running_loss_val / len(val_loader.dataset)
        epoch_acc_val = running_corrects_val.double() / len(val_loader.dataset)

        print(f"Train Loss: {epoch_loss_train:.4f} Train Acc: {epoch_acc_train:.4f}")
        print(
            f"Validation Loss: {epoch_loss_val:.4f} Validation Acc: {epoch_acc_val:.4f}"
        )

        early_stopping(epoch_loss_val, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    print(f"Finished fold {fold + 1}/{len(dataloaders)}")

print("Finished Training")

Starting fold 1/5
Starting epoch 1/10


100%|██████████| 563/563 [00:56<00:00,  9.88it/s] 


Train Loss: 0.8615 Train Acc: 0.6252
Validation Loss: 0.6970 Validation Acc: 0.7077
Validation loss decreased (inf --> 0.697021).  Saving model ...
Starting epoch 2/10


100%|██████████| 563/563 [00:58<00:00,  9.59it/s] 


Train Loss: 0.6708 Train Acc: 0.7191
Validation Loss: 0.5936 Validation Acc: 0.7535
Validation loss decreased (0.697021 --> 0.593599).  Saving model ...
Starting epoch 3/10


...

Train Loss: 0.4892 Train Acc: 0.7953
Validation Loss: 0.4591 Validation Acc: 0.8091
Validation loss decreased (0.476276 --> 0.459094).  Saving model ...
Starting epoch 9/10


100%|██████████| 563/563 [00:57<00:00,  9.85it/s] 


Train Loss: 0.4732 Train Acc: 0.8012
Validation Loss: 0.4535 Validation Acc: 0.8095
Validation loss decreased (0.459094 --> 0.453479).  Saving model ...
Starting epoch 10/10


100%|██████████| 563/563 [00:57<00:00,  9.86it/s] 

Train Loss: 0.4605 Train Acc: 0.8056
Validation Loss: 0.4353 Validation Acc: 0.8186
Validation loss decreased (0.453479 --> 0.435323).  Saving model ...
Finished fold 5/5
Finished Training


In [5]:
# Dicionário para armazenar os relatórios de classificação para cada fold
classification_reports = []

# Caminho para a pasta com os modelos
model_folder = f"models/{checkpoint_folder}"

# Lista para armazenar a acurácia média de cada fold
accuracy_list = []

# Loop através de cada fold
for fold in range(k_folds):
    # Carregando o modelo
    model_path = os.path.join(model_folder, f"checkpoint_{fold}.pth")
    model.load_state_dict(torch.load(model_path))

    # Pegando o dataloader de teste para este fold
    _, test_loader = dataloaders[fold]

    # Avaliando o modelo neste fold
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Calculando a acurácia para este fold
    fold_accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
    accuracy_list.append(fold_accuracy)

    # Gerando o relatório de classificação para este fold
    report = classification_report(all_labels, all_preds, output_dict=True)
    classification_reports.append(report)

# Calculando a média e desvio padrão das métricas para cada classe em todos os folds
metrics_list = {
    class_: {"precision": [], "recall": [], "f1-score": [], "support": []}
    for class_ in range(num_classes)
}

avg_metrics = metrics_list


for report in classification_reports:
    for class_ in range(num_classes):
        metrics_list[class_]["precision"].append(report[str(class_)]["precision"])
        metrics_list[class_]["recall"].append(report[str(class_)]["recall"])
        metrics_list[class_]["f1-score"].append(report[str(class_)]["f1-score"])
        metrics_list[class_]["support"].append(report[str(class_)]["support"])

for class_ in range(num_classes):
    avg_metrics[class_]["precision_std"] = np.std(avg_metrics[class_]["precision"])
    avg_metrics[class_]["recall_std"] = np.std(avg_metrics[class_]["recall"])
    avg_metrics[class_]["f1-score_std"] = np.std(avg_metrics[class_]["f1-score"])
    avg_metrics[class_]["support_std"] = np.round(
        np.std(avg_metrics[class_]["support"])
    ).astype(int)

    avg_metrics[class_]["precision"] = np.mean(metrics_list[class_]["precision"])
    avg_metrics[class_]["recall"] = np.mean(metrics_list[class_]["recall"])
    avg_metrics[class_]["f1-score"] = np.mean(metrics_list[class_]["f1-score"])
    avg_metrics[class_]["support"] = np.round(
        np.mean(metrics_list[class_]["support"])
    ).astype(int)

# Média e desvio padrão da acurácia
accuracy_mean = np.mean(accuracy_list)
accuracy_std = np.std(accuracy_list)


print("Acurácias:", accuracy_list)
print("Precisions:", [metrics_list[class_]["precision"] for class_ in range(num_classes)])
print("Recalls:", [metrics_list[class_]["recall"] for class_ in range(num_classes)])
print("f1-scores:", [metrics_list[class_]["f1-score"] for class_ in range(num_classes)])




Acurácias: [0.8125590441789385, 0.8251180883578771, 0.8145595998888581, 0.8277759253084361, 0.8185506279871068]
Precisions: [0.835632291540823, 0.678723294809296, 0.8294192174705026, 0.8346890396011286]
Recalls: [0.8346149096108615, 0.7075227391910582, 0.8248856956237753, 0.81604447974583]
f1-scores: [0.8350991582487299, 0.6924806054736624, 0.8268682837168628, 0.8237559769099473]


In [6]:
annotation_dict = {
    0: "Nada",
    1: "Grito",
    2: "Violência Física",
    3: "Tiro",
    4: "Explosão",
    5: "Grito de Esforço",
}


print("Acurácia média:", accuracy_mean)
print("Desvio padrão da acurácia:", accuracy_std)

print(
    "================================================================================"
)

for class_ in range(num_classes):
    print(f"Class: {annotation_dict[class_]}")
    print("Precision\t\tRecall\t\t\tF1-Score\t\tSupport")
    print(
        "{:.4f} ± {:.4f}\t\t{:.4f} ± {:.4f}\t\t{:.4f} ± {:.4f}\t\t{}".format(
            avg_metrics[class_]["precision"],
            avg_metrics[class_]["precision_std"],
            avg_metrics[class_]["recall"],
            avg_metrics[class_]["recall_std"],
            avg_metrics[class_]["f1-score"],
            avg_metrics[class_]["f1-score_std"],
            avg_metrics[class_]["support"],
        )
    )
    print(
        "================================================================================"
    )

Acurácia média: 0.8197126571442434
Desvio padrão da acurácia: 0.005887503195577952
Class: Nada
Precision		Recall			F1-Score		Support
0.8356 ± 0.0046		0.8346 ± 0.0105		0.8351 ± 0.0067		9154
Class: Grito
Precision		Recall			F1-Score		Support
0.6787 ± 0.0127		0.7075 ± 0.0227		0.6925 ± 0.0098		1457
Class: Violência Física
Precision		Recall			F1-Score		Support
0.8294 ± 0.0205		0.8249 ± 0.0115		0.8269 ± 0.0066		6124
Class: Tiro
Precision		Recall			F1-Score		Support
0.8347 ± 0.0367		0.8160 ± 0.0345		0.8238 ± 0.0069		1259
